<div class="alert alert-block alert-success">
     
# IC PCDA CAPSTONE - HARRY BAKHSHI - NOTEBOOK 9 - MAKE SEASONAL DATA STATIONARY
     
</div>

MIT License for code used from https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python in notebook:   
https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python?tab=MIT-1-ov-file     

(Accessed 06/06/2024)

In [168]:
import numpy as np
import matplotlib.pyplot as plt
import os
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"
import pandas as pd
import pickle
import statistics
pd.options.mode.chained_assignment = None  # default='warn'

Load timeseries and seasonal period list:

In [169]:
#Import timeseries data and seasonal period list:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/outlier_treated_ts_df_feature_engineered_2.pik'
# with open(file, 'wb') as f:
#        pickle.dump(ts_df, f) #write df to .pik file on disk
with open(file, 'rb') as f:
     ts_df = pickle.load(f) #load pickle file 'file' into variable
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/janatahack_demand_forecasting_data_variable_seasonal_period_list_2_2.pik'
# with open(file, 'wb') as f:
#     pickle.dump(seasonal_period_list, f) #write df to .pik file on disk
with open(file, 'rb') as f:
     seasonal_period_list = pickle.load(f) #load pickle file 'file' into variable   

In [170]:
ts_df[ts_df.columns.values[137:146]].head()

,weekday_name,weekday,week,day,date,month,month_name,year,week_of_month
timestamp,,,,,,,,,
2011-01-10,Monday,0,2,10,2011-01-10,1,January,2011,2.0
2011-01-17,Monday,0,3,17,2011-01-17,1,January,2011,4.0
2011-01-24,Monday,0,4,24,2011-01-24,1,January,2011,5.0
2011-01-31,Monday,0,5,31,2011-01-31,1,January,2011,6.0
2011-02-07,Monday,0,6,7,2011-02-07,2,February,2011,1.0


In [171]:
ts_df.drop(list(ts_df.columns.values[137:146]), axis=1, inplace=True)
ts_df.drop('timestamp', axis=1, inplace=True)

In [172]:
#list of variables in the seasonal period list:
name_list = list(ts_df.columns.values[:136])
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/name_list_2.pik'
with open(file, 'wb') as f:
    pickle.dump(name_list, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      name_list = pickle.load(f) #load pickle file 'file' into variable
print(name_list)

['679023median_total_price', '245338median_total_price', '222087median_total_price', '223153median_total_price', '223245median_total_price', '222765median_total_price', '547934median_total_price', '378934median_total_price', '219029median_total_price', '216418median_total_price', '219009median_total_price', '217390median_total_price', '216233median_total_price', '216425median_total_price', '320485median_total_price', '300021median_total_price', '245387median_total_price', '216419median_total_price', '398721median_total_price', '217217median_total_price', '217777median_total_price', '219844median_total_price', '600934median_total_price', '673209median_total_price', '327492median_total_price', '300291median_total_price', '546789median_total_price', '545621median_total_price', '679023median_base_price', '245338median_base_price', '222087median_base_price', '223153median_base_price', '223245median_base_price', '222765median_base_price', '547934median_base_price', '378934median_base_price',

Re-split feature engineered data:

In [173]:
#Do train-test-validation split: 
test = ts_df.iloc[133:, :]
val = ts_df.iloc[114:133, :]
train = ts_df.iloc[:114, :]
train_and_val = ts_df.iloc[:133, :]
print(f"# of Training samples: {len(train)} | # of Validation samples: {len(val)} | # of Test samples: {len(test)} | # of Train+val samples: {len(train_and_val)}")
print(f"Max Date in Train: {train.index.date.max()} | Min Date in Validation: {val.index.date.min()} | Min Date in Test: {test.index.date.min()} | Max Date in Train+val: {train_and_val.index.date.max()}| Min Date in Train+val: {train_and_val.index.date.min()}")
#^https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python/blob/main/notebooks/Chapter04/01-Setting%20up%20Experiment%20Harness.ipynb
#Accessed 17/06/2024

# of Training samples: 114 | # of Validation samples: 19 | # of Test samples: 19 | # of Train+val samples: 133
Max Date in Train: 2013-03-11 | Min Date in Validation: 2013-03-18 | Min Date in Test: 2013-07-29 | Max Date in Train+val: 2013-07-22| Min Date in Train+val: 2011-01-10


In [174]:
col_list = list(ts_df.columns.values[:])
seasonal_period_list
# col_list

[52,
 52,
 52,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 52,
 'not seasonal',
 52,
 'not seasonal',
 'not seasonal',
 52,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 52,
 'not seasonal',
 52,
 4,
 'not seasonal',
 52,
 'not seasonal',
 52,
 8,
 52,
 52,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 52,
 'not seasonal',
 'not seasonal',
 52,
 'not seasonal',
 52,
 'not seasonal',
 52,
 52,
 8,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 4,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 'not seasonal',
 'not seasonal',
 9,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 'not seasonal',
 8,
 4,
 'not seasonal',
 'not seasonal',
 'not seasonal',
 52,
 'not seasonal',
 52,
 'not seasonal',
 '

In [175]:
train.head()

,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,216418median_total_price,...,timestamp_Week_cos_1,timestamp_Week_cos_2,timestamp_Week_cos_3,timestamp_Week_cos_4,timestamp_Week_cos_5,timestamp_Week_cos_6,timestamp_Week_cos_7,timestamp_Week_cos_8,timestamp_Week_cos_9,timestamp_Week_cos_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-10,180.2625,469.537500,226.931250,213.037500,205.2000,195.225000,177.412500,205.912500,312.787500,87.637500,...,0.970942,0.885456,0.748511,0.568065,0.354605,0.120537,-0.120537,-0.354605,-0.568065,-0.748511
2011-01-17,178.1250,426.787500,196.650000,190.237500,205.9125,240.825000,177.412500,177.412500,312.075000,93.337500,...,0.935016,0.748511,0.464723,0.120537,-0.239316,-0.568065,-0.822984,-0.970942,-0.992709,-0.885456
2011-01-24,178.1250,426.787500,190.237500,190.237500,213.0375,240.825000,177.412500,177.412500,312.787500,85.500000,...,0.885456,0.568065,0.120537,-0.354605,-0.748511,-0.970942,-0.970942,-0.748511,-0.354605,0.120537
2011-01-31,178.1250,448.162500,226.931250,212.325000,213.0375,240.825000,177.412500,177.412500,312.787500,84.075000,...,0.822984,0.354605,-0.239316,-0.748511,-0.992709,-0.885456,-0.464723,0.120537,0.663123,0.970942
2011-02-07,180.2625,448.795621,175.185694,212.135054,205.2000,240.792458,177.600521,177.604805,245.708337,90.036567,...,0.748511,0.120537,-0.568065,-0.970942,-0.885456,-0.354605,0.354605,0.885456,0.970942,0.568065


# MAKE TRAINING DATA STATIONARY
- for predicting validation set

In [176]:
for col in range(len(col_list)):
    train[col_list[col]] = pd.to_numeric(train[col_list[col]])

In [177]:
# count = 0
# for col in range(len(col_list)):
#     for row in range(len(train)):
#         if pd.isnull(train[col_list[col]][row]):
#             count += 1
# print(count)

In [178]:
for col in range(len(col_list)):
    ser = train[col_list[col]].copy()
    drop_list = []
    for row in range(len(train)):
        if pd.isnull(ser.iloc[row]):
            drop_list.append(list(ser.index)[row])
    ser.drop(drop_list, inplace=True)
    ser = list(ser)
    ser_act = train[col_list[col]]
    for row in range(len(train)):
        if pd.isnull(ser_act.iloc[row]):
            if not ser:
                ser_act.iloc[row] = 0
            else:
                ser_act.iloc[row] = statistics.median(ser)

/tmp/ipykernel_96852/2171507813.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.07573007419705391' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  ser_act.iloc[row] = statistics.median(ser)


In [179]:
# count = 0
# for col in range(len(col_list)):
#     for row in range(len(train)):
#         if pd.isnull(train[col_list[col]][row]):
#             count += 1
# print(count)

In [180]:
for col in range(len(col_list)):
    val[col_list[col]] = pd.to_numeric(val[col_list[col]])
for col in range(len(col_list)):
    ser = val[col_list[col]].copy()
    drop_list = []
    for row in range(len(val)):
        if pd.isnull(ser.iloc[row]):
            drop_list.append(list(ser.index)[row])
    ser.drop(drop_list, inplace=True)
    ser = list(ser)
    ser_act = val[col_list[col]]
    for row in range(len(val)):
        if pd.isnull(ser_act.iloc[row]):
            if not ser:
                ser_act.iloc[row] = 0
            else:
                ser_act.iloc[row] = statistics.median(ser)

In [181]:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/train_feature_engineered_2.pik'
with open(file, 'wb') as f:
    pickle.dump(train, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      train = pickle.load(f) #load pickle file 'file' into variable
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/val_feature_engineered_2.pik'
with open(file, 'wb') as f:
    pickle.dump(val, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      val = pickle.load(f) #load pickle file 'file' into variable

In [182]:
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/train_feature_engineered_2.pik'
# # with open(file, 'wb') as f:
# #     pickle.dump(train, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      train = pickle.load(f) #load pickle file 'file' into variable

In [183]:
train.head()

,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,216418median_total_price,...,timestamp_Week_cos_1,timestamp_Week_cos_2,timestamp_Week_cos_3,timestamp_Week_cos_4,timestamp_Week_cos_5,timestamp_Week_cos_6,timestamp_Week_cos_7,timestamp_Week_cos_8,timestamp_Week_cos_9,timestamp_Week_cos_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-10,180.2625,469.537500,226.931250,213.037500,205.2000,195.225000,177.412500,205.912500,312.787500,87.637500,...,0.970942,0.885456,0.748511,0.568065,0.354605,0.120537,-0.120537,-0.354605,-0.568065,-0.748511
2011-01-17,178.1250,426.787500,196.650000,190.237500,205.9125,240.825000,177.412500,177.412500,312.075000,93.337500,...,0.935016,0.748511,0.464723,0.120537,-0.239316,-0.568065,-0.822984,-0.970942,-0.992709,-0.885456
2011-01-24,178.1250,426.787500,190.237500,190.237500,213.0375,240.825000,177.412500,177.412500,312.787500,85.500000,...,0.885456,0.568065,0.120537,-0.354605,-0.748511,-0.970942,-0.970942,-0.748511,-0.354605,0.120537
2011-01-31,178.1250,448.162500,226.931250,212.325000,213.0375,240.825000,177.412500,177.412500,312.787500,84.075000,...,0.822984,0.354605,-0.239316,-0.748511,-0.992709,-0.885456,-0.464723,0.120537,0.663123,0.970942
2011-02-07,180.2625,448.795621,175.185694,212.135054,205.2000,240.792458,177.600521,177.604805,245.708337,90.036567,...,0.748511,0.120537,-0.568065,-0.970942,-0.885456,-0.354605,0.354605,0.885456,0.970942,0.568065


In [184]:
#!pip install pymannkendall

Weak stationarity caused by constant mean, constant variance, autocorrelation constant (1)  (autocorrelation changes occur with periodic, seasonal behaviour (2))   
Deterministic trend, heteroscedasticity, stochastic trend, seasonality make a series non-stationary (2)   

To make variable stationary:
- does max = min? This is a case where all values are the same and so mean = constant, variance = constant, series is weakly stationary
- is trend deterministic (mean changes) (2) or stochastic (constant mean, variance changes; = unit root (2))?
- stochastic → stabilise mean by differencing (2)(3)(4), remove seasonality if present, remove heteroscedasticity (changes in variance)
- deterministic → stabilise mean by detrending (2)(5), remove seasonality if present, remove heteroscedasticity

(1) - https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc442.htm  
^Accessed 21/07/2024   
(2) Modern Time Series Forecasting with Python (Manu Joseph, 2022)   
(3) - https://blog.stata.com/2016/06/21/unit-root-tests-in-stata/     
^Accessed 21/07/2024   
(4) - https://otexts.com/fpp2/stationarity.html   
^Accessed 21/07/2024   
(5) - https://www.sciencedirect.com/science/article/abs/pii/016920709090003T  
^Accessed 21/07/2024   

In [185]:
from src.transforms.target_transformations import DeseasonalizingTransformer
def AutoStationaryTransformModified(df, col_list, var, index=None):
    #Creating the timeseries with datetime index
    y = df[col_list[var]]
    if y.max() != y.min():
        #Check for stochastic trend:
        if check_unit_root(y, confidence=0.05).stationary == False:
            diff_transformer = AdditiveDifferencingTransformer()
            # [1:] because differencing reduces the length of the time series by one
            y_diff = diff_transformer.fit_transform(y, freq="W-MON")[1:]
            # Setting the transformed series back to the dataframe
            df[col_list[var]][1:] = y_diff.values
            df[col_list[var]][0] = statistics.median(df[col_list[var]][1:])
            #Creating the timeseries with datetime index
            y = df[col_list[var]]
            if index != None:
                deseasonalizing_transformer = DeseasonalizingTransformer(seasonality_extraction="period_averages",seasonal_period=seasonal_period_list[index])
            else:
                deseasonalizing_transformer = DeseasonalizingTransformer(seasonality_extraction="period_averages",seasonal_period=seasonal_period_list[var])
            y_deseasonalized = deseasonalizing_transformer.fit_transform(y, freq="W-MON")
            #Setting the transformed series back to the dataframe           
            df[col_list[var]] = y_deseasonalized.values
            #Creating the timeseries with datetime index
            y = df[col_list[var]]
            if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                #shifting the series into positive domain
                _y_hetero = y-y.min()
                #Arbritarily divided the data into sub-series of length 25
                boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                y_diff = boxcox_transformer.fit_transform(_y_hetero)
                # Setting the transformed series back to the dataframe
                df[col_list[var]] = y_diff.values
            else:
                pass
        else:
            #Check deterministic trend
            if check_deterministic_trend(y, confidence=0.05).deterministic_trend == True:
            # if check_trend(y, confidence=0.05, seasonal_period=None, mann_kendall=True, prewhiten=None).trend == True:
                detrending_transformer = DetrendingTransformer()
                y_diff = detrending_transformer.fit_transform(y, freq="W-MON")
                # Setting the transformed series back to the dataframe
                df[col_list[var]] = y_diff.values
                #Creating the timeseries with datetime index
                y = df[col_list[var]]
                if index != None:
                    deseasonalizing_transformer = DeseasonalizingTransformer(seasonality_extraction="period_averages",seasonal_period=seasonal_period_list[index])
                else:
                    deseasonalizing_transformer = DeseasonalizingTransformer(seasonality_extraction="period_averages",seasonal_period=seasonal_period_list[var])
                y_deseasonalized = deseasonalizing_transformer.fit_transform(y, freq="W-MON")
                #Setting the transformed series back to the dataframe           
                df[col_list[var]] = y_deseasonalized.values
                #Creating the timeseries with datetime index
                y = df[col_list[var]]
                #Check heteroscedastisticity
                if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                    #shifting the series into positive domain
                    _y_hetero = y-y.min()
                    #Arbritarily divided the data into sub-series of length 25
                    boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                    y_diff = boxcox_transformer.fit_transform(_y_hetero)
                    # Setting the transformed series back to the dataframe
                    df[col_list[var]] = y_diff.values
            else:
                pass
    else:
        pass
    return df[col_list[var]]

In [186]:
#Make training data stationary:
# from src.transforms.target_transformations import AutoStationaryTransformer
from src.transforms.target_transformations import DetrendingTransformer
from src.transforms.stationary_utils import check_unit_root
from src.transforms.target_transformations import AdditiveDifferencingTransformer
from src.transforms.stationary_utils import check_trend
from src.transforms.stationary_utils import check_deterministic_trend
from src.transforms.stationary_utils import check_heteroscedastisticity
from src.transforms.target_transformations import BoxCoxTransformer
transformer_pipelines = {}
# for var in range(4):
for var in range(len(col_list)):
    if var < 136:
    # if var < 140:
        if seasonal_period_list[var] != 'not seasonal':
            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var)
            # #Initialize the AutoStationaryTransformer with seasonality period
            # auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period_list[var])
            # #Creating the timeseries with datetime index
            # y = train[col_list[var]]
            # #Fitting and trainsforming the train
            # y_stat = auto_stationary.fit_transform(y, freq="W-MON")
            # # Setting the transformerd series back to the dataframe           
            # train[col_list[var]] = y_stat.values
            # #Saving the pipeline
            # transformer_pipelines[col_list[var]] = auto_stationary
        if seasonal_period_list[var] == 'not seasonal':
            #Creating the timeseries with datetime index
            y = train[col_list[var]]
            #Check stationary
            if y.max() != y.min():
                #Check for stochastic trend:
                if check_unit_root(y, confidence=0.05).stationary == False:
                    diff_transformer = AdditiveDifferencingTransformer()
                    # [1:] because differencing reduces the length of the time series by one
                    y_diff = diff_transformer.fit_transform(y, freq="W-MON")[1:]
                    # Setting the transformed series back to the dataframe
                    train[col_list[var]][1:] = y_diff.values
                    train[col_list[var]][0] = statistics.median(train[col_list[var]][1:])
                    #Creating the timeseries with datetime index
                    y = train[col_list[var]]
                    if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                        #shifting the series into positive domain
                        _y_hetero = y-y.min()
                        #Arbritarily divided the data into sub-series of length 25
                        boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                        y_diff = boxcox_transformer.fit_transform(_y_hetero)
                        # Setting the transformed series back to the dataframe
                        train[col_list[var]] = y_diff.values
                    else:
                        pass
                else:
                    #Check deterministic trend
                    if check_deterministic_trend(y, confidence=0.05).deterministic_trend == True:
                    # if check_trend(y, confidence=0.05, seasonal_period=None, mann_kendall=True, prewhiten=None).trend == True:
                        detrending_transformer = DetrendingTransformer()
                        y_diff = detrending_transformer.fit_transform(y, freq="W-MON")
                        # Setting the transformed series back to the dataframe
                        train[col_list[var]] = y_diff.values
                        #Creating the timeseries with datetime index
                        y = train[col_list[var]]
                        #Check heteroscedastisticity
                        if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                            #shifting the series into positive domain
                            _y_hetero = y-y.min()
                            #Arbritarily divided the data into sub-series of length 25
                            boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                            y_diff = boxcox_transformer.fit_transform(_y_hetero)
                            # Setting the transformed series back to the dataframe
                            train[col_list[var]] = y_diff.values
                    else:
                        pass
            else:
                pass
        else:
            pass
    else:       
        found_output = col_list[var].find('seasonal')
        if found_output != -1:
            found_output = col_list[var].find('median_total_price')
            if found_output != -1:
                index = name_list.index(col_list[var][0:24])
                if seasonal_period_list[index] != 'not seasonal':
                    train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                    # #Initialize the AutoStationaryTransformer with seasonality period
                    # auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period_list[index])
                    # #Creating the timeseries with datetime index
                    # y = train[col_list[var]]
                    # #Fitting and trainsforming the train
                    # y_stat = auto_stationary.fit_transform(y, freq="W-MON")
                    # # Setting the transformerd series back to the dataframe
                    # train[col_list[var]] = y_stat.values
                    # #Saving the pipeline
                    # transformer_pipelines[col_list[var]] = auto_stationary
            found_output = col_list[var].find('median_base_price')
            if found_output != -1:
                index = name_list.index(col_list[var][0:23])
                if seasonal_period_list[index] != 'not seasonal':
                    train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
            found_output = col_list[var].find('median_units_sold')
            if found_output != -1:
                index = name_list.index(col_list[var][0:23])
                if seasonal_period_list[index] != 'not seasonal':
                    train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
            found_output = col_list[var].find('percent_featured')
            if found_output != -1:
                index = name_list.index(col_list[var][0:22])
                if seasonal_period_list[index] != 'not seasonal':
                    train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
            found_output = col_list[var].find('percent_display')
            if found_output != -1:
                index = name_list.index(col_list[var][0:21])
                if seasonal_period_list[index] != 'not seasonal':
                    train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
            else:
                found_output = col_list[var].find('ewma')
                if found_output != -1:
                    found_output = col_list[var].find('median_total_price')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:24])
                        if seasonal_period_list[index] != 'not seasonal':
                            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                    found_output = col_list[var].find('median_base_price')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:23])
                        if seasonal_period_list[index] != 'not seasonal':
                            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                    found_output = col_list[var].find('median_units_sold')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:23])
                        if seasonal_period_list[index] != 'not seasonal':
                            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                    found_output = col_list[var].find('percent_featured')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:22])
                        if seasonal_period_list[index] != 'not seasonal':
                            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                    found_output = col_list[var].find('percent_display')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:21])
                        if seasonal_period_list[index] != 'not seasonal':
                            train[col_list[var]] = AutoStationaryTransformModified(train, col_list, var, index)
                else:
                    #Creating the timeseries with datetime index
                    y = train[col_list[var]]
                    #Check stationary
                    if y.max() != y.min():
                        #Check for stochastic trend:
                        if check_unit_root(y, confidence=0.05).stationary == False:
                            diff_transformer = AdditiveDifferencingTransformer()
                            # [1:] because differencing reduces the length of the time series by one
                            y_diff = diff_transformer.fit_transform(y, freq="W-MON")[1:]
                            # Setting the transformed series back to the dataframe
                            train[col_list[var]][1:] = y_diff.values
                            train[col_list[var]][0] = statistics.median(train[col_list[var]][1:])
                            if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                                #shifting the series into positive domain
                                _y_hetero = y-y.min()
                                #Arbritarily divided the data into sub-series of length 25
                                boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                                y_diff = boxcox_transformer.fit_transform(_y_hetero)
                                # Setting the transformed series back to the dataframe
                                train[col_list[var]] = y_diff.values
                            else:
                                pass
                        else:
                            #Check deterministic trend
                            if check_deterministic_trend(y, confidence=0.05).deterministic_trend == True:
                            # if check_trend(y, confidence=0.05, seasonal_period=None, mann_kendall=True, prewhiten=None).trend == True:
                                detrending_transformer = DetrendingTransformer()
                                y_diff = detrending_transformer.fit_transform(y, freq="W-MON")
                                # Setting the transformed series back to the dataframe
                                train[col_list[var]] = y_diff.values
                                #Creating the timeseries with datetime index
                                y = train[col_list[var]]
                                #Check heteroscedastisticity
                                if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                                    #shifting the series into positive domain
                                    _y_hetero = y-y.min()
                                    #Arbritarily divided the data into sub-series of length 25
                                    boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                                    y_diff = boxcox_transformer.fit_transform(_y_hetero)
                                    # Setting the transformed series back to the dataframe
                                    train[col_list[var]] = y_diff.values
                            else:
                                pass
                    else:
                        pass
  
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/train_stationary_2.pik'
with open(file, 'wb') as f:
    pickle.dump(train, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      train = pickle.load(f) #load pickle file 'file' into variable
    
# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/auto_stationary_transformer_pipeline_train.pik'
# with open(file, 'wb') as f:
#     pickle.dump(transformer_pipelines, f) #write df to .pik file on disk
# # with open(file, 'rb') as f:
# #      transformer_pipelines = pickle.load(f) #load pickle file 'file' into variable

#^https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python/blob/main/notebooks/Chapter07/02-Dealing%20with%20Non-Stationarity.ipynb
#Accessed 04/07/2024

/tmp/ipykernel_96852/48967971.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df[col_list[var]][0] = statistics.median(df[col_list[var]][1:])
/tmp/ipykernel_96852/537860190.py:39: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  train[col_list[var]][0] = statistics.median(train[col_list[var]][1:])
/tmp/ipykernel_96852/537860190.py:39: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  train[col_list[var]][0] = statistics

In [187]:
train.head()

,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,216418median_total_price,...,timestamp_Week_cos_1,timestamp_Week_cos_2,timestamp_Week_cos_3,timestamp_Week_cos_4,timestamp_Week_cos_5,timestamp_Week_cos_6,timestamp_Week_cos_7,timestamp_Week_cos_8,timestamp_Week_cos_9,timestamp_Week_cos_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-10,180.2625,469.537500,5718.078564,0.000000,205.2000,195.225000,177.412500,205.912500,312.787500,87.637500,...,0.970942,0.885456,0.748511,0.568065,0.354605,0.120537,-0.120537,-0.354605,-0.568065,-0.748511
2011-01-17,178.1250,426.787500,0.000000,-22.800000,205.9125,240.825000,177.412500,177.412500,312.075000,93.337500,...,0.935016,0.748511,0.464723,0.120537,-0.239316,-0.568065,-0.822984,-0.970942,-0.992709,-0.885456
2011-01-24,178.1250,426.787500,1956.735899,0.000000,213.0375,240.825000,177.412500,177.412500,312.787500,85.500000,...,0.885456,0.568065,0.120537,-0.354605,-0.748511,-0.970942,-0.970942,-0.748511,-0.354605,0.120537
2011-01-31,178.1250,448.162500,4841.076303,22.087500,213.0375,240.825000,177.412500,177.412500,312.787500,84.075000,...,0.822984,0.354605,-0.239316,-0.748511,-0.992709,-0.885456,-0.464723,0.120537,0.663123,0.970942
2011-02-07,180.2625,448.795621,728.932091,-0.189946,205.2000,240.792458,177.600521,177.604805,245.708337,90.036567,...,0.748511,0.120537,-0.568065,-0.970942,-0.885456,-0.354605,0.354605,0.885456,0.970942,0.568065


# MAKE TRAIN+VAL DATA STATIONARY
- for predicting test set

In [188]:
for col in range(len(col_list)):
    train_and_val[col_list[col]] = pd.to_numeric(train_and_val[col_list[col]])

In [189]:
for col in range(len(col_list)):
    ser = train_and_val[col_list[col]].copy()
    drop_list = []
    for row in range(len(train_and_val)):
        if pd.isnull(ser.iloc[row]):
            drop_list.append(list(ser.index)[row])
    ser.drop(drop_list, inplace=True)
    ser = list(ser)
    ser_act = train_and_val[col_list[col]]
    for row in range(len(train_and_val)):
        if pd.isnull(ser_act.iloc[row]):
            if not ser:
                ser_act.iloc[row] = 0
            else:
                ser_act.iloc[row] = statistics.median(ser)

In [190]:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/train_and_val_feature_engineered.pik'
with open(file, 'wb') as f:
    pickle.dump(train_and_val, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      train_and_val = pickle.load(f) #load pickle file 'file' into variable

In [191]:
train_and_val.head()

,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,216418median_total_price,...,timestamp_Week_cos_1,timestamp_Week_cos_2,timestamp_Week_cos_3,timestamp_Week_cos_4,timestamp_Week_cos_5,timestamp_Week_cos_6,timestamp_Week_cos_7,timestamp_Week_cos_8,timestamp_Week_cos_9,timestamp_Week_cos_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-10,180.2625,469.537500,226.931250,213.037500,205.2000,195.225000,177.412500,205.912500,312.787500,87.637500,...,0.970942,0.885456,0.748511,0.568065,0.354605,0.120537,-0.120537,-0.354605,-0.568065,-0.748511
2011-01-17,178.1250,426.787500,196.650000,190.237500,205.9125,240.825000,177.412500,177.412500,312.075000,93.337500,...,0.935016,0.748511,0.464723,0.120537,-0.239316,-0.568065,-0.822984,-0.970942,-0.992709,-0.885456
2011-01-24,178.1250,426.787500,190.237500,190.237500,213.0375,240.825000,177.412500,177.412500,312.787500,85.500000,...,0.885456,0.568065,0.120537,-0.354605,-0.748511,-0.970942,-0.970942,-0.748511,-0.354605,0.120537
2011-01-31,178.1250,448.162500,226.931250,212.325000,213.0375,240.825000,177.412500,177.412500,312.787500,84.075000,...,0.822984,0.354605,-0.239316,-0.748511,-0.992709,-0.885456,-0.464723,0.120537,0.663123,0.970942
2011-02-07,180.2625,448.795621,175.185694,212.135054,205.2000,240.792458,177.600521,177.604805,245.708337,90.036567,...,0.748511,0.120537,-0.568065,-0.970942,-0.885456,-0.354605,0.354605,0.885456,0.970942,0.568065


Weak stationarity caused by constant mean, constant variance, autocorrelation constant (1)  (autocorrelation changes occur with periodic, seasonal behaviour (2))   
Deterministic trend, heteroscedasticity, stochastic trend, seasonality make a series non-stationary (2)   

To make variable stationary:
- does max = min? This is a case where all values are the same and so mean = constant, variance = constant, series is weakly stationary
- is trend deterministic (mean changes) (2) or stochastic (constant mean, variance changes; = unit root (2))?
- stochastic → stabilise mean by differencing (2)(3)(4), remove seasonality if present, remove heteroscedasticity (changes in variance)
- deterministic → stabilise mean by detrending (2)(5), remove seasonality if present, remove heteroscedasticity

(1) - https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc442.htm  
^Accessed 21/07/2024   
(2) Modern Time Series Forecasting with Python (Manu Joseph, 2022)   
(3) - https://blog.stata.com/2016/06/21/unit-root-tests-in-stata/     
^Accessed 21/07/2024   
(4) - https://otexts.com/fpp2/stationarity.html   
^Accessed 21/07/2024   
(5) - https://www.sciencedirect.com/science/article/abs/pii/016920709090003T  
^Accessed 21/07/2024   

In [192]:
#Make train+val data stationary:
# from src.transforms.target_transformations import AutoStationaryTransformer
from src.transforms.target_transformations import DetrendingTransformer
from src.transforms.stationary_utils import check_unit_root
from src.transforms.target_transformations import AdditiveDifferencingTransformer
from src.transforms.stationary_utils import check_trend
from src.transforms.stationary_utils import check_deterministic_trend
from src.transforms.stationary_utils import check_heteroscedastisticity
from src.transforms.target_transformations import BoxCoxTransformer
transformer_pipelines = {}
for var in range(len(col_list)):
    if var < 136:
    # if var < 140:
        if seasonal_period_list[var] != 'not seasonal':
            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var)
            # #Initialize the AutoStationaryTransformer with seasonality period
            # auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period_list[var])
            # #Creating the timeseries with datetime index
            # y = train_and_val[col_list[var]]
            # #Fitting and trainsforming the train
            # y_stat = auto_stationary.fit_transform(y, freq="W-MON")
            # # Setting the transformerd series back to the dataframe           
            # train_and_val[col_list[var]] = y_stat.values
            # #Saving the pipeline
            # transformer_pipelines[col_list[var]] = auto_stationary
        if seasonal_period_list[var] == 'not seasonal':
            #Creating the timeseries with datetime index
            y = train_and_val[col_list[var]]
            #Check stationary
            if y.max() != y.min():
                #Check for stochastic trend:
                if check_unit_root(y, confidence=0.05).stationary == False:
                    diff_transformer = AdditiveDifferencingTransformer()
                    # [1:] because differencing reduces the length of the time series by one
                    y_diff = diff_transformer.fit_transform(y, freq="W-MON")[1:]
                    # Setting the transformed series back to the dataframe
                    train_and_val[col_list[var]][1:] = y_diff.values
                    train_and_val[col_list[var]][0] = statistics.median(train_and_val[col_list[var]][1:])
                    #Creating the timeseries with datetime index
                    y = train_and_val[col_list[var]]
                    if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                        #shifting the series into positive domain
                        _y_hetero = y-y.min()
                        #Arbritarily divided the data into sub-series of length 25
                        boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                        y_diff = boxcox_transformer.fit_transform(_y_hetero)
                        # Setting the transformed series back to the dataframe
                        train_and_val[col_list[var]] = y_diff.values
                    else:
                        pass
                else:
                    #Check deterministic trend
                    if check_deterministic_trend(y, confidence=0.05).deterministic_trend == True:
                    # if check_trend(y, confidence=0.05, seasonal_period=None, mann_kendall=True, prewhiten=None).trend == True:
                        detrending_transformer = DetrendingTransformer()
                        y_diff = detrending_transformer.fit_transform(y, freq="W-MON")
                        # Setting the transformed series back to the dataframe
                        train_and_val[col_list[var]] = y_diff.values
                        #Creating the timeseries with datetime index
                        y = train_and_val[col_list[var]]
                        #Check heteroscedastisticity
                        if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                            #shifting the series into positive domain
                            _y_hetero = y-y.min()
                            #Arbritarily divided the data into sub-series of length 25
                            boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                            y_diff = boxcox_transformer.fit_transform(_y_hetero)
                            # Setting the transformed series back to the dataframe
                            train_and_val[col_list[var]] = y_diff.values
                    else:
                        pass
            else:
                pass
        else:
            pass
    else:       
        found_output = col_list[var].find('seasonal')
        if found_output != -1:
            found_output = col_list[var].find('median_total_price')
            if found_output != -1:
                index = name_list.index(col_list[var][0:24])
                if seasonal_period_list[index] != 'not seasonal':
                    train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                    # #Initialize the AutoStationaryTransformer with seasonality period
                    # auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period_list[index])
                    # #Creating the timeseries with datetime index
                    # y = train_and_val[col_list[var]]
                    # #Fitting and trainsforming the train
                    # y_stat = auto_stationary.fit_transform(y, freq="W-MON")
                    # # Setting the transformerd series back to the dataframe
                    # train_and_val[col_list[var]] = y_stat.values
                    # #Saving the pipeline
                    # transformer_pipelines[col_list[var]] = auto_stationary
            found_output = col_list[var].find('median_base_price')
            if found_output != -1:
                index = name_list.index(col_list[var][0:23])
                if seasonal_period_list[index] != 'not seasonal':
                    train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
            found_output = col_list[var].find('median_units_sold')
            if found_output != -1:
                index = name_list.index(col_list[var][0:23])
                if seasonal_period_list[index] != 'not seasonal':
                    train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
            found_output = col_list[var].find('percent_featured')
            if found_output != -1:
                index = name_list.index(col_list[var][0:22])
                if seasonal_period_list[index] != 'not seasonal':
                    train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
            found_output = col_list[var].find('percent_display')
            if found_output != -1:
                index = name_list.index(col_list[var][0:21])
                if seasonal_period_list[index] != 'not seasonal':
                    train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
            else:
                found_output = col_list[var].find('ewma')
                if found_output != -1:
                    found_output = col_list[var].find('median_total_price')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:24])
                        if seasonal_period_list[index] != 'not seasonal':
                            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                    found_output = col_list[var].find('median_base_price')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:23])
                        if seasonal_period_list[index] != 'not seasonal':
                            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                    found_output = col_list[var].find('median_units_sold')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:23])
                        if seasonal_period_list[index] != 'not seasonal':
                            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                    found_output = col_list[var].find('percent_featured')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:22])
                        if seasonal_period_list[index] != 'not seasonal':
                            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                    found_output = col_list[var].find('percent_display')
                    if found_output != -1:
                        index = name_list.index(col_list[var][0:21])
                        if seasonal_period_list[index] != 'not seasonal':
                            train_and_val[col_list[var]] = AutoStationaryTransformModified(train_and_val, col_list, var, index)
                else:
                    #Creating the timeseries with datetime index
                    y = train_and_val[col_list[var]]
                    #Check stationary
                    if y.max() != y.min():
                        #Check for stochastic trend:
                        if check_unit_root(y, confidence=0.05).stationary == False:
                            diff_transformer = AdditiveDifferencingTransformer()
                            # [1:] because differencing reduces the length of the time series by one
                            y_diff = diff_transformer.fit_transform(y, freq="W-MON")[1:]
                            # Setting the transformed series back to the dataframe
                            train_and_val[col_list[var]][1:] = y_diff.values
                            train_and_val[col_list[var]][0] = statistics.median(train_and_val[col_list[var]][1:])
                            #Creating the timeseries with datetime index
                            y = train_and_val[col_list[var]]
                            if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                                #shifting the series into positive domain
                                _y_hetero = y-y.min()
                                #Arbritarily divided the data into sub-series of length 25
                                boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                                y_diff = boxcox_transformer.fit_transform(_y_hetero)
                                # Setting the transformed series back to the dataframe
                                train_and_val[col_list[var]] = y_diff.values
                            else:
                                pass
                        else:
                            #Check deterministic trend
                            if check_deterministic_trend(y, confidence=0.05).deterministic_trend == True:
                            # if check_trend(y, confidence=0.05, seasonal_period=None, mann_kendall=True, prewhiten=None).trend == True:
                                detrending_transformer = DetrendingTransformer()
                                y_diff = detrending_transformer.fit_transform(y, freq="W-MON")
                                # Setting the transformed series back to the dataframe
                                train_and_val[col_list[var]] = y_diff.values
                                #Creating the timeseries with datetime index
                                y = train_and_val[col_list[var]]
                                #Check heteroscedastisticity
                                if check_heteroscedastisticity(y, confidence=0.005)[0] == True:
                                    #shifting the series into positive domain
                                    _y_hetero = y-y.min()
                                    #Arbritarily divided the data into sub-series of length 25
                                    boxcox_transformer = BoxCoxTransformer(seasonal_period=25, add_one=True, optimization="guerrero")
                                    y_diff = boxcox_transformer.fit_transform(_y_hetero)
                                    # Setting the transformed series back to the dataframe
                                    train_and_val[col_list[var]] = y_diff.values
                            else:
                                pass
                    else:
                        pass
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/train_and_val_stationary_2.pik'
with open(file, 'wb') as f:
    pickle.dump(train_and_val, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      train_and_val = pickle.load(f) #load pickle file 'file' into variable

# file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/auto_stationary_transformer_pipeline_train_and_val_2.pik'
# with open(file, 'wb') as f:
#     pickle.dump(transformer_pipelines, f) #write df to .pik file on disk
# # with open(file, 'rb') as f:
# #      transformer_pipelines = pickle.load(f) #load pickle file 'file' into variable

#^https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python/blob/main/notebooks/Chapter07/02-Dealing%20with%20Non-Stationarity.ipynb
#^https://github.com/PacktPublishing/Modern-Time-Series-Forecasting-with-Python/blob/main/notebooks/Chapter07/02a-Dealing%20with%20Non-Stationarity-Train%2BVal.ipynb
#Accessed 04/07/2024

/tmp/ipykernel_96852/48967971.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df[col_list[var]][0] = statistics.median(df[col_list[var]][1:])
/tmp/ipykernel_96852/1691313460.py:38: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  train_and_val[col_list[var]][0] = statistics.median(train_and_val[col_list[var]][1:])
/tmp/ipykernel_96852/48967971.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df[col_list[var]][0]

In [193]:
train_and_val.head()

,679023median_total_price,245338median_total_price,222087median_total_price,223153median_total_price,223245median_total_price,222765median_total_price,547934median_total_price,378934median_total_price,219029median_total_price,216418median_total_price,...,timestamp_Week_cos_1,timestamp_Week_cos_2,timestamp_Week_cos_3,timestamp_Week_cos_4,timestamp_Week_cos_5,timestamp_Week_cos_6,timestamp_Week_cos_7,timestamp_Week_cos_8,timestamp_Week_cos_9,timestamp_Week_cos_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2011-01-10,180.2625,469.537500,0.990670,0.000000,205.2000,195.225000,177.412500,205.912500,1.522737,87.637500,...,0.970942,0.885456,0.748511,0.568065,0.354605,0.120537,-0.120537,-0.354605,-0.568065,-0.748511
2011-01-17,178.1250,426.787500,0.000000,-22.800000,205.9125,240.825000,177.412500,177.412500,0.000000,93.337500,...,0.935016,0.748511,0.464723,0.120537,-0.239316,-0.568065,-0.822984,-0.970942,-0.992709,-0.885456
2011-01-24,178.1250,426.787500,0.984020,0.000000,213.0375,240.825000,177.412500,177.412500,1.484744,85.500000,...,0.885456,0.568065,0.120537,-0.354605,-0.748511,-0.970942,-0.970942,-0.748511,-0.354605,0.120537
2011-01-31,178.1250,448.162500,0.989841,22.087500,213.0375,240.825000,177.412500,177.412500,1.484353,84.075000,...,0.822984,0.354605,-0.239316,-0.748511,-0.992709,-0.885456,-0.464723,0.120537,0.663123,0.970942
2011-02-07,180.2625,448.795621,0.973822,-0.189946,205.2000,240.792458,177.600521,177.604805,1.418130,90.036567,...,0.748511,0.120537,-0.568065,-0.970942,-0.885456,-0.354605,0.354605,0.885456,0.970942,0.568065


Pickle test data:

In [194]:
for col in range(len(col_list)):
    test[col_list[col]] = pd.to_numeric(test[col_list[col]])

In [195]:
for col in range(len(col_list)):
    ser = test[col_list[col]].copy()
    drop_list = []
    for row in range(len(test)):
        if pd.isnull(ser.iloc[row]):
            drop_list.append(list(ser.index)[row])
    ser.drop(drop_list, inplace=True)
    ser = list(ser)
    ser_act = test[col_list[col]]
    for row in range(len(test)):
        if pd.isnull(ser_act.iloc[row]):
            if not ser:
                ser_act.iloc[row] = 0
            else:
                ser_act.iloc[row] = statistics.median(ser)

In [196]:
file = '/Users/harrybakhshi/Desktop/Python_notes/data/PCDA_capstone/test_feature_engineered_2.pik'
with open(file, 'wb') as f:
    pickle.dump(test, f) #write df to .pik file on disk
# with open(file, 'rb') as f:
#      test = pickle.load(f) #load pickle file 'file' into variable